In [96]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [97]:
movies = pd.read_csv('IMDb movies.csv')

C:\Users\veere\.conda\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [98]:
movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [99]:
movies = movies[['genre', 'actors', 'director', 'production_company', 'writer', 'title']].fillna('')

In [100]:

train_data = movies['title'] + ', ' + movies['genre'].replace(" ", ",", regex=True) + ', ' + movies['actors'] + ', ' + movies['director']+ ', '  + movies['production_company']+ ', '  + movies['writer']

In [101]:
train_data

0        Miss Jerry, Romance, Blanche Bayliss, William ...
1        The Story of the Kelly Gang, Biography,,Crime,...
2        Den sorte drøm, Drama, Asta Nielsen, Valdemar ...
3        Cleopatra, Drama,,History, Helen Gardner, Pear...
4        L'Inferno, Adventure,,Drama,,Fantasy, Salvator...
                               ...                        
85850    Le lion, Comedy, Dany Boon, Philippe Katerine,...
85851    De Beentjes van Sint-Hildegard, Comedy,,Drama,...
85852    Padmavyuhathile Abhimanyu, Drama, Anoop Chandr...
85853    Sokagin Çocuklari, Drama,,Family, Ahmet Faik A...
85854    La vida sense la Sara Amat, Drama, Maria Morer...
Length: 85855, dtype: object

In [102]:
def tokenize(text):
    text = str(text).split(',')
    text = [str(word).strip() for word in text if str(word).strip()!='']
    return text
    
cv = CountVectorizer(tokenizer=tokenize, ngram_range=(2,2), max_features=1000)#tokenizer=tokenize

In [108]:
train = cv.fit_transform(train_data[:50000])

In [109]:
cv.get_feature_names()

['aav creative unit godfrey ho',
 'abbas alibhai burmawalla mastan alibhai burmawalla',
 'abby berlin columbia pictures',
 'action adventure',
 'action biography',
 'action comedy',
 'action crime',
 'action drama',
 'action fantasy',
 'action history',
 'action horror',
 'action music',
 'action mystery',
 'action romance',
 'action sci-fi',
 'action sport',
 'action thriller',
 'action war',
 'action western',
 'adriano bolzoni adriano bolzoni',
 'adventure action',
 'adventure animation',
 'adventure biography',
 'adventure comedy',
 'adventure crime',
 'adventure drama',
 'adventure family',
 'adventure fantasy',
 'adventure history',
 'adventure horror',
 'adventure johnny sheffield',
 'adventure johnny weissmuller',
 'adventure lex barker',
 'adventure masako nozawa',
 'adventure music',
 'adventure mystery',
 'adventure romance',
 'adventure sci-fi',
 'adventure thriller',
 'adventure war',
 'adventure western',
 'agenore incrocci furio scarpelli',
 'akira kurosawa toho company'

In [110]:
movies['title']

0                            Miss Jerry
1           The Story of the Kelly Gang
2                        Den sorte drøm
3                             Cleopatra
4                             L'Inferno
                      ...              
85850                           Le lion
85851    De Beentjes van Sint-Hildegard
85852         Padmavyuhathile Abhimanyu
85853                 Sokagin Çocuklari
85854        La vida sense la Sara Amat
Name: title, Length: 85855, dtype: object

In [111]:
from sklearn.metrics.pairwise import cosine_similarity

In [112]:
cosine_sim = cosine_similarity(train)

In [113]:
df=movies.copy()

In [114]:
df.index

RangeIndex(start=0, stop=85855, step=1)

In [115]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title].index.values[0]

In [116]:
movie_user_likes = "Miss Jerry"#"Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it

In [117]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:6]

In [118]:
sorted_similar_movies

[(1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]

In [119]:
i=0
print("Top 5 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

Top 5 similar movies to Miss Jerry are:

The Story of the Kelly Gang
Den sorte drøm
Cleopatra
L'Inferno
From the Manger to the Cross; or, Jesus of Nazareth
